## Hyperpolarization-Activated Channels

### 电生理特性

超极化激活通道，是一类在膜电位超极化时被激活的阳离子通道。它们在心脏节律神经元和中枢神经系统中的某些类型神经元中发挥着关键的调节作用：

- 调节静息膜电位：在静息状态下产生的去极化电流可稳定神经元的膜电位，防止过度超极化。
- 参与节律性放电：在丘脑等结构中，帮助神经元维持固有节律放电。
- 兴奋性调节：能够降低神经元的输入电阻，从而降低细胞对输入的敏感性。

### 建模实现

由于我们在 `braincell` 中没有内置很多的超极化激活通道，所以我们直接让具体的超极化激活通道继承自 `Channel` 类，这在实际使用上并不会有任何影响。

下面我们给出一个超极化激活通道的例子。

`Ih_HM1992` 是由 Huguenard & McCormick 在 1992 年提出的超极化激活阳离子电流模型，其动力学由下式给出：

$$
\begin{aligned}
        I_h &= g_{\mathrm{max}} p \\
        \frac{dp}{dt} &= \phi \frac{p_{\infty} - p}{\tau_p} \\
        p_{\infty} &=\frac{1}{1+\exp ((V+75) / 5.5)} \\
        \tau_{p} &=\frac{1}{\exp (-0.086 V-14.59)+\exp (0.0701 V-1.87)}
        \end{aligned}
        $$

其中：
- $p$ 是激活变量
- $g_{\text{max}}$ 是最大电导
- $E$ 是反转电位（约 43 mV）
- $\phi$ 是温度因子

具体代码实现如下：

```python
class Ih_HM1992(Channel):
    def __init__(
        self,
        size,
        g_max=10. * (u.mS / u.cm ** 2),
        E=43. * u.mV,
        phi=1.,
        name=None,
    ):
        super().__init__(size=size, name=name)
        self.phi = brainstate.init.param(phi, self.varshape, allow_none=False)
        self.g_max = brainstate.init.param(g_max, self.varshape, allow_none=False)
        self.E = brainstate.init.param(E, self.varshape, allow_none=False)

    def init_state(self, V, batch_size=None):
        self.p = DiffEqState(brainstate.init.param(u.math.zeros, self.varshape, batch_size))

    def reset_state(self, V, batch_size=None):
        self.p.value = self.f_p_inf(V)

    def compute_derivative(self, V):
        self.p.derivative = self.phi * (self.f_p_inf(V) - self.p.value) / self.f_p_tau(V) / u.ms

    def current(self, V):
        return self.g_max * self.p.value * (self.E - V)

    def f_p_inf(self, V):
        V = V.to_decimal(u.mV)
        return 1. / (1. + u.math.exp((V + 75.) / 5.5))

    def f_p_tau(self, V):
        V = V.to_decimal(u.mV)
        return 1. / (u.math.exp(-0.086 * V - 14.59) + u.math.exp(0.0701 * V - 1.87))
```

通过这个例子你可以看到，超极化激活通道的建模非常自然地融入 `braincell` 框架，借助 `Channel` 的接口，轻松表达其独特的动力学性质。